In [63]:
import pandas as pd

In [64]:
movies=pd.read_csv("tmdb_5000_movies.csv")

In [65]:
credits=pd.read_csv("tmdb_5000_credits.csv")

In [66]:
new_movies=movies.merge(credits,on="title")

In [67]:
new_movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [68]:
#genre
#id
#keyword
#overview
#title
#cast
#crew these are the only columns we need

In [69]:
new_movies=new_movies[['id','title','genres','keywords','overview','cast','crew']]
new_movies.head(1)

,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [70]:
new_movies.dropna(inplace=True)

In [71]:
import json

In [72]:
def formatgenre(text):
    genres=[]
    for i in json.loads(text):
        genres.append(i["name"])
    return genres

In [73]:
new_movies['genres']=new_movies['genres'].apply(formatgenre)

In [74]:
def formatkeywords(text):
    keywords=[]
    for i in json.loads(text):
        keywords.append(i["name"])
    return keywords

In [75]:
new_movies['keywords']=new_movies['keywords'].apply(formatkeywords)

In [76]:
def castformatter(text):
    cast=[]
    z=0
    for i in json.loads(text):
        if z==3:
            break
        z=z+1
        cast.append(i["name"])
    return cast

In [77]:
new_movies['cast']=new_movies['cast'].apply(castformatter)

In [78]:
def crewformatter(text):
    director=[]
    for i in json.loads(text):
        if i["job"]=="Director":
            director.append(i["name"])
            break
    return director

In [79]:
new_movies['crew']=new_movies['crew'].apply(crewformatter)

In [80]:
new_movies.head()

,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",Following the death of District Attorney Harve...,"[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","John Carter is a war-weary, former military ca...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


In [81]:
new_movies["keywords"]=new_movies["keywords"]+new_movies["genres"]+new_movies["cast"]

In [82]:
new_movies["keywords"][0]

['culture clash',
 'future',
 'space war',
 'space colony',
 'society',
 'space travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love affair',
 'anti war',
 'power relations',
 'mind and soul',
 '3d',
 'Action',
 'Adventure',
 'Fantasy',
 'Science Fiction',
 'Sam Worthington',
 'Zoe Saldana',
 'Sigourney Weaver']